In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the previously saved model
model_path = 'sin_predictor.h5'  # Make sure to have the correct path
model = load_model(model_path)

# Retrieve input shape from the model
input_shape = model.input_shape[1:]  # This excludes the batch size dimension

# Representative dataset generator
def representative_data_gen():
    for _ in range(100):  # Adjust the number of samples as needed
        # Generate a sample using the input shape. Replace with real data if available.
        yield [np.random.random((1,) + input_shape).astype(np.float32)]

# Set up the converter for int8 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
tflite_quant_model = converter.convert()

# Save the TFLite model to a file
with open('sin_predictor.tflite', 'wb') as f:
    f.write(tflite_quant_model)

print("Model converted to TFLite with int8 quantization and saved successfully!")


Model converted to TFLite with int8 quantization and saved successfully!


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
# Load the TFLite model
tflite_model_path = 'sin_predictor.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)

# Allocate tensors to the interpreter
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input and output details
print("Input details:")
for input_detail in input_details:
    print(f"Name: {input_detail['name']}, Shape: {input_detail['shape']}, Type: {input_detail['dtype']}")

print("\nOutput details:")
for output_detail in output_details:
    print(f"Name: {output_detail['name']}, Shape: {output_detail['shape']}, Type: {output_detail['dtype']}")


Input details:
Name: serving_default_input_1:0, Shape: [1 7], Type: <class 'numpy.int8'>

Output details:
Name: StatefulPartitionedCall:0, Shape: [1 1], Type: <class 'numpy.int8'>


In [ ]:
!xxd sin_predictor.tflite sin_predictor.cc